<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4f8fda9be688dc99169450800cf279743ce3f80d0a740f4bb05c61cfe7d42884
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-22 13:09:07
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  3.11 L
Current:  1.43 C
-------------------
Today PnL: -34.80 K (-0.24%)
Current PnL: -17.09 L (-11.5%)
CY Booked + Current PnL: -5.65 L (-3.8%)
-------------------
Total profit:  2.00 L
Total loss:  -19.09 L
-------------------
Total Booked + Current PnL: 21.33 L (17.53%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.0%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.26 C
Est FTT PnL: 82.61 L (57.77%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.93%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,15.60,69.0,H-MC,3.52,199470.0,34800.0,8537.0,1.57,21.13,4.28,26.31,89.0,4.08,1.43,30.80,X40N,NTT,AC
76,TTKPRESTIG,770.00,100.21,47.0,M-SC,3.32,86270.0,-14507.0,14597.0,-0.87,-14.40,16.92,0.09,245.0,-0.99,0.62,13.34,OX40N,NTT,DURABLES
49,NATIONALUM,244.55,-43.66,66.0,H-MC,8.59,112455.0,11951.0,17161.0,0.27,11.89,15.26,28.96,79.0,0.70,0.80,49.90,MH,ATH,METALS
56,RELIANCE,1533.00,-12.16,55.0,H-LC,4.83,219430.0,9064.0,19727.0,-0.06,4.31,8.99,13.68,37.0,0.46,1.57,21.54,XY25,NTT,REFINERIES
41,ITC,452.00,-38.34,46.0,H-LC,2.28,199316.0,-822.0,21706.0,-0.74,-0.41,10.89,10.44,4.0,-0.04,1.42,5.13,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1276.40,81.0,M-SC,6.50,187193.0,6050.0,39423.0,12.28,3.34,21.06,25.10,235.0,0.15,1.34,50.02,XY24,NTT,HEALTHCARE
8,AWL,485.00,-63.12,66.0,H-MC,3.88,262291.0,-39460.0,200889.0,6.89,-13.08,76.59,53.50,116.0,-0.20,1.88,16.50,XY24,NTT,FMCG
80,VAIBHAVGBL,521.00,64.74,66.0,H-SC,6.50,142760.0,-40015.0,151083.0,2.34,-21.89,105.83,60.77,125.0,-0.26,1.02,27.86,XR,NTT,JEWELLERY
31,HINDZINC,730.22,23.40,64.0,M-LC,1.69,201448.0,-3628.0,116195.0,2.22,-1.77,57.68,54.89,52.0,-0.03,1.44,21.90,X5K,ATH,METALS
34,ICICIPRULI,790.00,-19.39,54.0,H-MC,2.16,138764.0,2976.0,39770.0,1.90,2.19,28.66,31.48,107.0,0.07,0.99,14.58,X40,ATH,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,-4.05,51.0,H-LC,1.44,238095.0,-8056.0,87262.0,-3.97,-3.27,36.65,32.18,16.0,-0.09,1.70,31.59,X200,ATH,IT
53,RAJESHEXPO,518.0,1800.25,53.0,L-SC,2.44,51858.0,-85319.0,85410.0,-3.38,-62.20,164.70,0.07,267.0,-1.00,0.37,28.97,OX40N,NTT,JEWELLERY
83,VOLTAS,1530.0,-1.22,48.0,H-MC,2.75,207150.0,15408.0,22351.0,-2.74,8.04,10.79,19.69,99.0,0.69,1.48,15.38,XY25,NTT,AC
39,INFY,2275.0,-18.16,49.0,H-LC,8.19,319394.0,6506.0,165191.0,-2.64,2.08,51.72,54.87,3.0,0.04,2.28,8.87,X40,BTT,IT
21,COFFEEDAY,80.0,-43.66,49.0,L-SC,42.32,83136.0,-30413.0,70466.0,-2.30,-26.78,84.76,35.27,268.0,-0.43,0.59,103.48,XR,NTT,HOTELS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,IEX,219.00,-29.46,55.0,H-SC,16.78,201284.0,1330.0,95892.0,-0.47,0.67,47.64,48.63,115.0,0.01,1.44,12.29,XR,NTT,MISC
29,HAVELLS,2069.16,0.48,59.0,H-MC,1.94,249226.0,1495.0,73572.0,0.43,0.60,29.52,30.30,92.0,0.02,1.78,13.98,X40,ATH,ELECTRICAL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,93.70,53.0,H-SC,9.32,122715.0,-19764.0,39281.0,-0.74,-13.87,32.01,13.70,163.0,-0.50,0.88,44.84,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,100.21,47.0,M-SC,3.32,86270.0,-14507.0,14597.0,-0.87,-14.40,16.92,0.09,245.0,-0.99,0.62,13.34,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.06,62.0,H-SC,0.81,226953.0,-42714.0,79048.0,0.13,-15.84,34.83,13.47,138.0,-0.54,1.62,14.79,XY24,NTT,PAINTS
18,CERA,9475.0,-20.53,45.0,H-SC,2.15,145015.0,-30888.0,72914.0,0.65,-17.56,50.28,23.89,149.0,-0.42,1.04,25.24,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-21.19,47.0,H-MC,7.26,106628.0,-25207.0,67432.0,-1.93,-19.12,63.24,32.03,98.0,-0.37,0.76,19.43,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.0,15.6,69.0,H-MC,3.52,199470.0,34800.0,8537.0,1.57,21.13,4.28,26.31,89.0,4.08,1.43,30.8,X40N,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1276.40,81.0,M-SC,6.50,187193.0,6050.0,39423.0,12.28,3.34,21.06,25.10,235.0,0.15,1.34,50.02,XY24,NTT,HEALTHCARE
59,SAIL,228.00,49.16,72.0,M-MC,8.23,235931.0,10969.0,156682.0,0.59,4.88,66.41,74.53,192.0,0.07,1.69,38.70,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,15.60,69.0,H-MC,3.52,199470.0,34800.0,8537.0,1.57,21.13,4.28,26.31,89.0,4.08,1.43,30.80,X40N,NTT,AC
29,HAVELLS,2069.16,0.48,59.0,H-MC,1.94,249226.0,1495.0,73572.0,0.43,0.60,29.52,30.30,92.0,0.02,1.78,13.98,X40,ATH,ELECTRICAL
17,CAMS,5211.76,-0.73,61.0,H-SC,3.08,112952.0,10948.0,32982.0,-1.63,10.73,29.20,43.06,122.0,0.33,0.81,29.69,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-43.66,66.0,H-MC,8.59,112455.0,11951.0,17161.0,0.27,11.89,15.26,28.96,79.0,0.70,0.80,49.90,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.79,50.0,M-SC,0.83,100035.0,8536.0,70215.0,-0.30,9.33,70.19,86.07,223.0,0.12,0.72,45.27,XR,NTT,DURABLES
11,BANKINDIA,190.00,-27.95,68.0,H-MC,6.62,186098.0,6290.0,106113.0,0.44,3.50,57.02,62.52,88.0,0.06,1.33,38.78,XR,NTT,BANKS
36,INDIAMART,4850.92,-53.52,32.0,H-SC,11.35,129749.0,6413.0,127349.0,-0.87,5.20,98.15,108.45,119.0,0.05,0.93,28.72,AR,ATH,MISC
85,WIPRO,420.00,-11.52,49.0,M-LC,5.68,155372.0,4427.0,105031.0,-2.06,2.93,67.60,72.51,53.0,0.04,1.11,9.13,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
30,HCLTECH,1943.91,-1.08,41.0,H-LC,9.84,225829.0,-16095.0,79356.0,-1.98,-6.65,35.14,26.15,8.0,-0.20,1.61,6.72,X40,ATH,IT
82,VBL,671.64,-16.73,41.0,H-LC,6.82,300051.0,-15791.0,128452.0,-0.87,-5.00,42.81,35.67,5.0,-0.12,2.14,8.16,X40N,ATH,FMCG
41,ITC,452.00,-38.34,46.0,H-LC,2.28,199316.0,-822.0,21706.0,-0.74,-0.41,10.89,10.44,4.0,-0.04,1.42,5.13,X40,NTT,FMCG
19,CIPLA,1795.00,-20.47,48.0,H-LC,5.52,212528.0,8028.0,33388.0,-1.55,3.93,15.71,20.25,10.0,0.24,1.52,13.00,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-12.93,48.0,H-LC,7.76,219622.0,-32146.0,85367.0,0.56,-12.77,38.87,21.13,27.0,-0.38,1.57,18.13,X40,ATH,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,-4.05,51.0,H-LC,1.44,238095.0,-8056.0,87262.0,-3.97,-3.27,36.65,32.18,16.0,-0.09,1.70,31.59,X200,ATH,IT
41,ITC,452.0,-38.34,46.0,H-LC,2.28,199316.0,-822.0,21706.0,-0.74,-0.41,10.89,10.44,4.0,-0.04,1.42,5.13,X40,NTT,FMCG
64,SIEMENS,4671.5,2.31,59.0,H-LC,3.56,164350.0,-21745.0,69224.0,-1.17,-11.68,42.12,25.51,15.0,-0.31,1.17,21.11,AR,ATH,ELECTRICAL
50,NESTLEIND,1377.0,-9.11,53.0,H-LC,4.64,279770.0,14344.0,42441.0,0.09,5.40,15.17,21.40,11.0,0.34,2.00,13.61,XY25,NTT,FMCG
56,RELIANCE,1533.0,-12.16,55.0,H-LC,4.83,219430.0,9064.0,19727.0,-0.06,4.31,8.99,13.68,37.0,0.46,1.57,21.54,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,ITC,452.00,-38.34,46.0,H-LC,2.28,199316.0,-822.0,21706.0,-0.74,-0.41,10.89,10.44,4.0,-0.04,1.42,5.13,X40,NTT,FMCG
73,TCS,4476.75,-25.15,58.0,H-LC,12.29,297905.0,-48051.0,122916.0,-0.24,-13.89,41.26,21.64,1.0,-0.39,2.13,5.53,X40,ATH,IT
79,UNITDSPR,1644.00,-11.52,58.0,H-LC,7.63,231908.0,-2236.0,50857.0,1.51,-0.95,21.93,20.77,86.0,-0.04,1.66,5.90,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-1.08,41.0,H-LC,9.84,225829.0,-16095.0,79356.0,-1.98,-6.65,35.14,26.15,8.0,-0.20,1.61,6.72,X40,ATH,IT
82,VBL,671.64,-16.73,41.0,H-LC,6.82,300051.0,-15791.0,128452.0,-0.87,-5.00,42.81,35.67,5.0,-0.12,2.14,8.16,X40N,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,2.31,59.0,H-LC,3.56,164350.0,-21745.0,69224.0,-1.17,-11.68,42.12,25.51,15.0,-0.31,1.17,21.11,AR,ATH,ELECTRICAL
41,ITC,452.00,-38.34,46.0,H-LC,2.28,199316.0,-822.0,21706.0,-0.74,-0.41,10.89,10.44,4.0,-0.04,1.42,5.13,X40,NTT,FMCG
19,CIPLA,1795.00,-20.47,48.0,H-LC,5.52,212528.0,8028.0,33388.0,-1.55,3.93,15.71,20.25,10.0,0.24,1.52,13.00,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-12.93,48.0,H-LC,7.76,219622.0,-32146.0,85367.0,0.56,-12.77,38.87,21.13,27.0,-0.38,1.57,18.13,X40,ATH,PAINTS
56,RELIANCE,1533.00,-12.16,55.0,H-LC,4.83,219430.0,9064.0,19727.0,-0.06,4.31,8.99,13.68,37.0,0.46,1.57,21.54,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.15,58.0,H-LC,12.29,297905.0,-48051.0,122916.0,-0.24,-13.89,41.26,21.64,1.0,-0.39,2.13,5.53,X40,ATH,IT
39,INFY,2275.00,-18.16,49.0,H-LC,8.19,319394.0,6506.0,165191.0,-2.64,2.08,51.72,54.87,3.0,0.04,2.28,8.87,X40,BTT,IT
41,ITC,452.00,-38.34,46.0,H-LC,2.28,199316.0,-822.0,21706.0,-0.74,-0.41,10.89,10.44,4.0,-0.04,1.42,5.13,X40,NTT,FMCG
82,VBL,671.64,-16.73,41.0,H-LC,6.82,300051.0,-15791.0,128452.0,-0.87,-5.00,42.81,35.67,5.0,-0.12,2.14,8.16,X40N,ATH,FMCG
1,ABB,7934.00,-37.34,69.0,H-LC,9.37,261734.0,115.0,119089.0,0.29,0.04,45.50,45.57,7.0,0.00,1.87,11.37,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TRIDENT,48.00,-3.11,69.0,M-SC,1.86,75335.0,-16920.0,43227.0,1.26,-18.34,57.38,28.51,224.0,-0.39,0.54,28.91,XR,NTT,TEXTILES
6,ASIANTILES,137.00,7105.56,68.0,L-SC,5.18,79081.0,-14729.0,91346.0,0.98,-15.70,115.51,81.67,269.0,-0.16,0.57,53.14,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-43.66,49.0,L-SC,42.32,83136.0,-30413.0,70466.0,-2.30,-26.78,84.76,35.27,268.0,-0.43,0.59,103.48,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-173.86,68.0,M-SC,3.94,85506.0,-16704.0,110098.0,-0.49,-16.34,128.76,91.37,208.0,-0.15,0.61,38.80,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-5.18,45.0,H-SC,12.31,88245.0,-12718.0,107253.0,-1.24,-12.60,121.54,93.63,148.0,-0.12,0.63,31.44,SR,ATH,CHEMICALS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1276.40,81.0,M-SC,6.50,187193.0,6050.0,39423.0,12.28,3.34,21.06,25.10,235.0,0.15,1.34,50.02,XY24,NTT,HEALTHCARE
70,TANLA,1963.11,-15.40,73.0,H-SC,13.45,226930.0,-43831.0,373776.0,-1.61,-16.19,164.71,121.86,133.0,-0.12,1.62,77.52,AR,ATH,IT
60,SAMMAANCAP,326.00,-173.86,68.0,M-SC,3.94,85506.0,-16704.0,110098.0,-0.49,-16.34,128.76,91.37,208.0,-0.15,0.61,38.80,XY25,NTT,FINANCE
80,VAIBHAVGBL,521.00,64.74,66.0,H-SC,6.50,142760.0,-40015.0,151083.0,2.34,-21.89,105.83,60.77,125.0,-0.26,1.02,27.86,XR,NTT,JEWELLERY
7,ATULAUTO,844.00,3748.65,57.0,M-SC,4.95,119947.0,-27421.0,79237.0,-1.14,-18.61,66.06,35.16,236.0,-0.35,0.86,22.51,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.70
1,25,44.24
2,50,75.36


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.85
LC    31.16
MC    24.01
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.00
X40      14.97
X40N     11.63
XR       11.47
AR        8.55
XY25      8.39
OX40N     7.96
X200      1.70
MH        1.67
X5K       1.44
SR        1.24
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.89
H-LC    24.59
H-MC    21.02
M-SC    13.43
M-LC     5.52
M-MC     2.69
L-SC     1.53
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.15,-14.12,76.34
FMCG,12.42,-3.24,40.18
FINANCE,10.20,-15.09,65.30
MISC,7.20,-14.24,77.75
BANKS,6.19,-15.22,75.82
PAINTS,5.73,-15.25,32.43
ELECTRICAL,5.62,-4.67,41.39
INSURANCE,3.75,-5.53,44.98
AC,3.62,4.20,11.84


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3048078.0,22
XR,1306825.0,14
AR,1134102.0,9
X40,752899.0,10
X40N,579556.0,9
OX40N,534692.0,10
XY25,373395.0,6
SR,247472.0,2
X5K,116195.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3614540.0,29
M-SC,1369564.0,17
H-MC,1165862.0,15
H-LC,1085080.0,14
M-LC,378302.0,4
M-MC,302065.0,2
L-SC,247222.0,3
L-MC,60208.0,1
L-LC,37660.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1212224.0      6
M-SC       XY24       782987.0      7
H-SC       AR         782156.0      5
           XR         760219.0      7
H-MC       XY24       533622.0      4
H-LC       X40        474536.0      5
M-MC       XY24       302065.0      2
H-SC       X40N       295183.0      4
           OX40N      254420.0      4
           SR         247472.0      2
H-LC       X40N       212697.0      3
H-MC       X40        206389.0      4
H-LC       AR         188313.0      2
M-SC       AR         163633.0      2
H-MC       XY25       163469.0      2
L-SC       XR         161812.0      2
M-LC       XY24       157076.0      2
M-SC       OX40N      127430.0      4
M-LC       X5K        116195.0      1
M-SC       XR         113442.0      2
           XY25       110098.0      1
H-MC       XR         106113.0      1
M-LC       XR         105031.0      1
H-LC       X200        87262.0      1
L-SC       OX40N       85410.0      1
M-SC       X40         71974.0      1
H-MC       X40N        71676.0      2
           OX40N       67432.0      1
H-SC       MH          62866.0      1
H-LC       XY25        62168.0      2
L-MC       XR          60208.0      1
H-LC       XY24        60104.0      1
L-LC       XY25        37660.0      1
H-MC       MH          17161.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 54.0 seconds
